In [1]:
%load_ext autoreload
%autoreload 2

import yaml
import torch
import matplotlib.pyplot as plt

In [2]:
config_file = './configs/lego.yaml'
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)

In [3]:
from utils.DataLoaderLego import DataLoaderLego

data_loader = DataLoaderLego(config['data'])

images, poses, focal = data_loader.getDataset('train')

In [8]:
device = 'cpu'
if torch.cuda.is_available():
    device = torch.cuda.current_device()
device = 'cpu'

In [9]:
from utils.Camera import Camera

cameras = []
H, W = images[0].shape[:2]

for i in range(images.shape[0]):
    cameras.append(Camera(H, W, poses[i], focal, device))

In [10]:
from utils.Renderer import Renderer

renderer = Renderer(config['renderer'])

In [12]:
ray_o, ray_dir = cameras[0].getRays()

_,_, z_samples = renderer.getSparsePoints(ray_o, ray_dir, True)
z_samples = z_samples.reshape((-1,64))
renderer.getFinePoints(ray_o,ray_dir,z_samples,torch.ones(z_samples.shape))

(tensor([[[-0.0499,  3.8346,  1.2085,  1.5318, -1.2300],
          [-0.0338,  3.7891,  1.2104,  1.5318, -1.2300],
          [-0.0443,  3.8186,  1.2092,  1.5318, -1.2300],
          ...,
          [-0.0451,  3.8210,  1.2091,  1.5318, -1.2300],
          [-0.0493,  3.8327,  1.2086,  1.5318, -1.2300],
          [-0.0212,  3.7537,  1.2119,  1.5318, -1.2300]],
 
         [[-0.0424,  3.8126,  1.2094,  1.5317, -1.2361],
          [-0.0237,  3.7590,  1.2117,  1.5317, -1.2361],
          [-0.0342,  3.7891,  1.2104,  1.5317, -1.2361],
          ...,
          [-0.0501,  3.8349,  1.2085,  1.5317, -1.2361],
          [-0.0377,  3.7992,  1.2100,  1.5317, -1.2361],
          [-0.0380,  3.8002,  1.2100,  1.5317, -1.2361]],
 
         [[-0.0312,  3.7791,  1.2108,  1.5316, -1.2422],
          [-0.0258,  3.7633,  1.2115,  1.5316, -1.2422],
          [-0.0439,  3.8166,  1.2093,  1.5316, -1.2422],
          ...,
          [-0.0380,  3.7992,  1.2100,  1.5316, -1.2422],
          [-0.0536,  3.8448,  1.2081,

In [17]:
from models.Nerf import Nerf
from utils.TrainerNerf import TrainerNerf
import imageio.v3 as iio

test_img = "nerf_synthetic/lego/val/r_0.png"
test_img = iio.imread(test_img)[:,:,:3]
test_img = torch.tensor(test_img)
test_img = torch.nn.functional.interpolate(test_img[None,...].permute(
            0, 3, 1, 2), (100, 100), mode='bilinear', align_corners=False).permute(0, 2, 3, 1)[0]
test_img = test_img/255

test_pose = torch.tensor([
                [
                    -0.963964581489563,
                    -0.2611401677131653,
                    0.0507759265601635,
                    0.2046843022108078
                ],
                [
                    0.26603081822395325,
                    -0.9462433457374573,
                    0.18398693203926086,
                    0.7416750192642212
                ],
                [
                    7.450580596923828e-09,
                    0.1908649355173111,
                    0.9816163182258606,
                    3.957021951675415
                ],
                [
                    0.0,
                    0.0,
                    0.0,
                    1.0
                ]
            ])

model_sparse = Nerf(config['model']['L_xyz'],config['model']['L_dir'])
model_fine = Nerf(config['model']['L_xyz'],config['model']['L_dir'])
model = (model_sparse, model_fine)
trainer = TrainerNerf(model, device, images, cameras, renderer, config['trainer'])
trainer.train(test_img[None,...], test_pose[None,...], focal)

9
9


KeyError: 'gamma'